In [1]:
import keras.backend as K
import multiprocessing
import tensorflow as tf
import pandas as pd
import numpy as np

from math import ceil

from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam

from keras.preprocessing.text import text_to_word_sequence

import os.path
import gc

from time import gmtime, strftime

import seaborn as sns
import matplotlib.pyplot as plt

gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


0

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
 
stop_words = set(stopwords.words('english')) 

def remove_stop_words(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

[nltk_data] Downloading package stopwords to C:\Users\Mike del
[nltk_data]     Castillo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Mike del
[nltk_data]     Castillo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
use_gpu = True

config = tf.ConfigProto(intra_op_parallelism_threads=multiprocessing.cpu_count(), 
                        inter_op_parallelism_threads=multiprocessing.cpu_count(), 
                        allow_soft_placement=True, 
                        device_count = {'CPU' : 1, 
                                        'GPU' : 1 if use_gpu else 0})

session = tf.Session(config=config)
K.set_session(session)

In [4]:
data = pd.read_csv('data/twitter-airline-sentiment.csv')

In [6]:
data['text'] = list(map(remove_stop_words, data['text'].values))
data.head()

,text,pos,neg
0,said,0.0,0.0
1,plus youve added commercials experience tacky,1.0,0.0
2,didnt today must mean need take another trip,0.0,0.0
3,really aggressive blast obnoxious entertainmen...,0.0,1.0
4,really big bad thing,0.0,1.0


In [8]:
corpus = [text_to_word_sequence(y) for y in [x[0] for x in data[['text']].values]]
labels = [np.array(x[[0, 1]]) for x in data[['pos', 'neg']].values]
    
print('Corpus size: {}'.format(len(corpus)))

Corpus size: 13871


In [20]:
max_sentence_length = 35
vector_size = 300

In [30]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

word2vec = Word2Vec(sentences=corpus,
                    size=vector_size, 
                    window=10, 
                    negative=20,
                    iter=50,
                    seed=1000,
                    workers=multiprocessing.cpu_count())

vecs_x = word2vec.wv
gc.collect()

222

In [31]:
def pad_vec_data(corpus):
    gc.collect()
    input_matrix = np.zeros((len(corpus), max_sent_length, vector_size), dtype=K.floatx())
    for i in range(len(corpus)):
        for t, token in enumerate(corpus[i]):
            if t >= max_sent_length:
                break
            if token not in vecs_x:
                continue
            input_matrix[i, t, :] = vecs_x[token]
    return input_matrix

In [35]:
# Keras convolutional model
gc.collect()
batch_size = 16
nb_epochs = 20

model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same', input_shape=(max_sentence_length, vector_size)))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Dropout(0.25))

model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

In [36]:
gc.collect()

train_x, test_x, train_y, test_y = train_test_split(pad_vec_data(corpus), labels)

train_x = np.array(train_x)
test_x = np.array(test_x)
train_y = np.array(train_y)
test_y = np.array(test_y)
gc.collect()

history = model.fit(train_x, train_y,
          batch_size=batch_size,
          shuffle=True,
          epochs=nb_epochs,
          validation_data=(test_x, test_y),
#                   verbose=0,
          callbacks=[
#                       EarlyStopping(min_delta=0.000025, patience=10),
          ])    

Train on 10403 samples, validate on 3468 samples
Epoch 1/20
10403/10403 [==============================] - 8s 724us/step - loss: 0.2771 - acc: 0.7115 - val_loss: 0.2116 - val_acc: 0.7416
Epoch 2/20
10403/10403 [==============================] - 6s 607us/step - loss: 0.2280 - acc: 0.7369 - val_loss: 0.2169 - val_acc: 0.7486
Epoch 3/20
10403/10403 [==============================] - 6s 612us/step - loss: 0.2163 - acc: 0.7456 - val_loss: 0.2089 - val_acc: 0.7442
Epoch 4/20
10403/10403 [==============================] - 6s 606us/step - loss: 0.2047 - acc: 0.7502 - val_loss: 0.2098 - val_acc: 0.7537
Epoch 5/20
10403/10403 [==============================] - 6s 607us/step - loss: 0.1963 - acc: 0.7533 - val_loss: 0.2100 - val_acc: 0.7399
Epoch 6/20
10403/10403 [==============================] - 6s 605us/step - loss: 0.1865 - acc: 0.7563 - val_loss: 0.2132 - val_acc: 0.7512
Epoch 7/20
10403/10403 [==============================] - 6s 597us/step - loss: 0.1766 - acc: 0.7610 - val_loss: 0.2200 - v

In [38]:
mess = [
    "hello there, my name is mike",
    "not something i wanted to happen but i think i need to add more words for it to see",
    "this was something i wanted i'm grateful for this app and i am looking forward to more things from you guys",
    "i don't fucking like you man",
    "considering how the usual process for this is tedious, having an app like this solves a lot of problems",
    "thank you that was some wonderful service that really helped me get from point A to point B",
    "this is amazing and was able to shorten the amount of time i needed to take to achieve this",
    "this is nothing short of amazing, I cannot believe it",
    "how are you",
    "i really like you",
    "i don't actually like this product!",
    "it's useless if you can't use it!",
    "i can't believe i never heard of this",
    "this is awesome, didn't know I needed this",
    "so what am i supposed to use this for?",
    "what do I need this for?",
    "this is good",
    "this is not good",
    "although the movie was great, it lacked impact",
    "the movie wasnt that nice",
    "the movie was nice",
    "this is not acceptable, I lost everything using your app",
    "that was kinda stupid",
    "the instructions were unlear and is not friendly for non-techy people",
    "this is really useful i would definitely tell everyone about it",
    "i need to try this!",
    "which is your favourite harry potter filmsorcerers stonechamber of secretsprisoner of azkabangoblet of firei like them all equallyi hate harry potter and think this is a stupid question",
    "sitting in the third row of the imax cinema at sydney s darling harbour  but i sometimes felt as though i was in the tiny two seater plane that carried the giant camera around australia  sweeping and gliding  banking and hovering over some of the most not",
]

pred = model.predict(pad_vec_data(list(map(remove_stop_words, mess))))
output = ''

for i ,m in enumerate(mess):
    output += ('{} {} {}\n'.format('POSITIVE:' if pred[i][0] > 0.5 else 'NEGATIVE:', m, pred[i]))

print(output)

NEGATIVE: hello there, my name is mike [0.07703983 0.9229602 ]
POSITIVE: not something i wanted to happen but i think i need to add more words for it to see [0.55461526 0.44538477]
POSITIVE: this was something i wanted i'm grateful for this app and i am looking forward to more things from you guys [0.5571791  0.44282088]
NEGATIVE: i don't fucking like you man [0.20254856 0.7974514 ]
POSITIVE: considering how the usual process for this is tedious, having an app like this solves a lot of problems [0.997552   0.00244804]
POSITIVE: thank you that was some wonderful service that really helped me get from point A to point B [0.6481324  0.35186765]
POSITIVE: this is amazing and was able to shorten the amount of time i needed to take to achieve this [0.9417186  0.05828135]
POSITIVE: this is nothing short of amazing, I cannot believe it [0.9563106  0.04368946]
NEGATIVE: how are you [0.25976652 0.7402335 ]
NEGATIVE: i really like you [0.00593156 0.9940685 ]
NEGATIVE: i don't actually like this p